In [1]:
import numpy as np
from utils import ClassEncoder
from datasets import get_iris_dataset
X_full_iris, y_full_iris = get_iris_dataset()
print(X_full_iris.shape)
print(y_full_iris.shape)

(150, 4)
(150, 1)


In [2]:
def print_distribution(_y):
    encoder = ClassEncoder()
    encoded_y = encoder.fit_transform(_y) # convert class to number (encode)
    print('Distribución de clases:')
    distribution = np.bincount(encoded_y.flatten())/len(encoded_y)
    for class_name, value in zip(encoder.names, distribution):
        print(f'{class_name}: {value:.4f}')

print_distribution(y_full_iris)


Distribución de clases:
setosa: 0.3333
versicolor: 0.3333
virginica: 0.3333


Se observa que el dataset Iris se encuentra balanceado, es decir que no hay alguna preponderancia de alguna de las clases por sobre las demás.

### 1) QDA Entrenado con:  probabilidades a priori uniforme y  una clase con probabilidad 0.9, las demás 0.05 ( 3 combinaciones)

In [3]:
from utils import split_transpose, QDA, accuracy

def priori_test(dataset):
    X_full, y_full = dataset
    a_priori_A = [1/3, 1/3, 1/3] # modelo 0
    a_priori_B_1 = [0.9, 0.05, 0.05] # modelo 1
    a_priori_B_2 = [0.05, 0.9, 0.05] # modelo 2
    a_priori_B_3 = [0.05, 0.05, 0.9] # modelo 3
    
    a_priori_list = [a_priori_A, a_priori_B_1, a_priori_B_2, a_priori_B_3]
    # from utils import QDA
    # rng_seed = 6543
    train_x, train_y, test_x, test_y = split_transpose(X_full, y_full, 0.4, 6543)
    
    for i,_a_priori in enumerate(a_priori_list):
        model = QDA()
        model.fit(train_x, train_y, _a_priori)
        print('A prioris:')
        print(",".join([f' {class_name}:{p:.3f} ' for class_name, p in zip(model.encoder.names, _a_priori)]))
        train_acc = accuracy(train_y, model.predict(train_x))
        test_acc = accuracy(test_y, model.predict(test_x))
        print(f"[Model {i}] Train (apparent) error is {1-train_acc:.4f} while test error is {1-test_acc:.4f}")
        # print('\n')

priori_test(get_iris_dataset())

A prioris:
 setosa:0.333 , versicolor:0.333 , virginica:0.333 
[Model 0] Train (apparent) error is 0.0222 while test error is 0.0167
A prioris:
 setosa:0.900 , versicolor:0.050 , virginica:0.050 
[Model 1] Train (apparent) error is 0.0222 while test error is 0.0167
A prioris:
 setosa:0.050 , versicolor:0.900 , virginica:0.050 
[Model 2] Train (apparent) error is 0.0333 while test error is 0.0000
A prioris:
 setosa:0.050 , versicolor:0.050 , virginica:0.900 
[Model 3] Train (apparent) error is 0.0333 while test error is 0.0500


A partir de estos datos y dejando de lado cuál es la verdadera distribución, se podrían hacer las siguientes suposiciones:
- El modelo 0 y el modelo 1 parecerían cometer el mismo grado de error al hacer dichas suposiciones sobre los priors (la clase setosa no parece verse afectada).
- El modelo 2 parecería sobreajustar (hay overfitting) a los datos de test.
- El modelo 3 tiene un mayor error tanto en el entrenamiento como en la prueba, lo que indica que no logra generalizar y que dichos priors tienen un efecto detrimental en la performance del modelo.
- Vemos que en los casos 2 y 3 aumenta el error en train y esto podria tener que ver con que la priori de setosa es baja.

Se utilizará la versión de QDA provista por SKLearn (a pesar de las diferencias en su implementación) a modo de evaluar cualitativamente el efecto de los priors sobre éste dataset y confirmar las suposiciones ya mencionadas, utilizando Cross Validation:

In [4]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import cross_val_score

def priori_test_cross_val(dataset):
    X_full, y_full = dataset
    a_priori_A = [1/3, 1/3, 1/3] # modelo 0
    a_priori_B_1 = [0.9, 0.05, 0.05] # modelo 1
    a_priori_B_2 = [0.05, 0.9, 0.05] # modelo 2
    a_priori_B_3 = [0.05, 0.05, 0.9] # modelo 3
        
    a_priori_list = [a_priori_A, a_priori_B_1, a_priori_B_2, a_priori_B_3]
    
    for i, _priors in enumerate(a_priori_list):
        qda = QuadraticDiscriminantAnalysis(priors=_priors) #utilizo QDA de sklearn para poder utilizar cross_val_score sin problemas
        cv_scores = cross_val_score(qda, X_full, y_full.flatten(), cv=5)  # 5-fold cross-validation
        print(f"Accuracy with priors {_priors}: {cv_scores.mean()} ± {cv_scores.std()} for model {i}")

priori_test_cross_val(get_iris_dataset())

Accuracy with priors [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]: 0.9800000000000001 ± 0.02666666666666666 for model 0
Accuracy with priors [0.9, 0.05, 0.05]: 0.9800000000000001 ± 0.02666666666666666 for model 1
Accuracy with priors [0.05, 0.9, 0.05]: 0.9733333333333334 ± 0.024944382578492935 for model 2
Accuracy with priors [0.05, 0.05, 0.9]: 0.9666666666666668 ± 0.036514837167011066 for model 3


Aquí se ve que el resultado del modelo 2 anterior no era representativo. La precisión es ligeramente menor que en los modelos anteriores, lo que podría indicar que las muestras de la segunda clase no son tan fácilmente separables como las de la primera clase.

En cuanto al modelo 3: presenta la precisión más baja y la desviación estándar más alta, lo que podría sugerir que la tercera clase es la más difícil de clasificar correctamente o que el sesgo hacia esa clase introduce más incertidumbre en las predicciones.

Con respecto al modelo 0 y 1: cabe la posibilidad de pensar que por ejemplo, las características que definen la primera clase son muy distintas, el modelo podría seguir clasificando correctamente la mayoría de las instancias de esa clase, incluso si los priors cambian. Ello querría decir que dicha clase es fácilmente separable de las demás. Para obtener una visión más clara de qué sucediendo entre el modelo 1 y el 0, se puede ver la matriz de confusión.

In [5]:
from sklearn.metrics import confusion_matrix

def get_cm(model, test_x, test_y):
    test_preds_0 = model.predict(test_x)
    return confusion_matrix(test_y.T, test_preds_0.T)


X_full, y_full = get_iris_dataset()
a_priori_A = [1/3, 1/3, 1/3] # modelo 0
a_priori_B_1 = [0.9, 0.05, 0.05] # modelo 1
a_priori_B_2 = [0.05, 0.9, 0.05] # modelo 2
a_priori_B_3 = [0.05, 0.05, 0.9] # modelo 3

train_x, train_y, test_x, test_y = split_transpose(X_full, y_full, 0.4, 6543)

model = QDA()

model.fit(train_x, train_y, a_priori_A)
print("Matriz de confusión de prueba para el modelo 0:")
print(get_cm(model, test_x, test_y))

model.fit(train_x, train_y, a_priori_B_1)
print("Matriz de confusión de prueba para el modelo 1:")
print(get_cm(model, test_x, test_y))

Matriz de confusión de prueba para el modelo 0:
[[23  0  0]
 [ 0 20  1]
 [ 0  0 16]]
Matriz de confusión de prueba para el modelo 1:
[[23  0  0]
 [ 0 20  1]
 [ 0  0 16]]


El hecho de que el modelo 1, con priors sesgados ([0.9, 0.05, 0.05]), produzca una matriz de confusión similar indica que la información proporcionada por los datos (en este caso de la clase 1) es lo suficientemente fuerte como para compensar el sesgo de los priors.


## 2) Repetir punto 1 para el dataset penguin

In [6]:
from datasets import get_penguins
X_full_penguin, y_full_penguin = get_penguins()

print_distribution(y_full_penguin)

Distribución de clases:
Adelie: 0.4415
Chinstrap: 0.1988
Gentoo: 0.3596


Se puede observar este dataset no está balanceado con respecto a la cantidad de datos por clase.

In [7]:
priori_test(get_penguins())

A prioris:
 Adelie:0.333 , Chinstrap:0.333 , Gentoo:0.333 
[Model 0] Train (apparent) error is 0.0098 while test error is 0.0073
A prioris:
 Adelie:0.900 , Chinstrap:0.050 , Gentoo:0.050 
[Model 1] Train (apparent) error is 0.0195 while test error is 0.0219
A prioris:
 Adelie:0.050 , Chinstrap:0.900 , Gentoo:0.050 
[Model 2] Train (apparent) error is 0.0098 while test error is 0.0219
A prioris:
 Adelie:0.050 , Chinstrap:0.050 , Gentoo:0.900 
[Model 3] Train (apparent) error is 0.0098 while test error is 0.0073


Los modelos que mejor generalizan son el 0 y el 3. Parece ser un caso análogo al de Iris en cuanto a lo que sucede con diferentes priors. Se intentará verificar a continuación si esto es así:

In [8]:
priori_test_cross_val(get_penguins())

Accuracy with priors [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]: 0.9882779198635976 ± 0.010993807100854342 for model 0
Accuracy with priors [0.9, 0.05, 0.05]: 0.9824381926683717 ± 0.005925745287640952 for model 1
Accuracy with priors [0.05, 0.9, 0.05]: 0.9589514066496164 ± 0.025287380244108995 for model 2
Accuracy with priors [0.05, 0.05, 0.9]: 0.9882779198635976 ± 0.010993807100854342 for model 3


Se puede ahora notar nuevamente que los modelos 1 (aunque por muy poco) y 2 son los que peor performan.

In [9]:
X_full, y_full = get_penguins()

train_x, train_y, test_x, test_y = split_transpose(X_full, y_full, 0.4, 6543)

model = QDA()

model.fit(train_x, train_y, a_priori_A)
print("Matriz de confusión de prueba para el modelo 0:")
print(get_cm(model, test_x, test_y))

model.fit(train_x, train_y, a_priori_B_1)
print("Matriz de confusión de prueba para el modelo 1:")
print(get_cm(model, test_x, test_y))

model.fit(train_x, train_y, a_priori_B_2)
print("Matriz de confusión de prueba para el modelo 2:")
print(get_cm(model, test_x, test_y))

model.fit(train_x, train_y, a_priori_B_3)
print("Matriz de confusión de prueba para el modelo 3:")
print(get_cm(model, test_x, test_y))

Matriz de confusión de prueba para el modelo 0:
[[67  0  0]
 [ 1 28  0]
 [ 0  0 41]]
Matriz de confusión de prueba para el modelo 1:
[[67  0  0]
 [ 3 26  0]
 [ 0  0 41]]
Matriz de confusión de prueba para el modelo 2:
[[64  3  0]
 [ 0 29  0]
 [ 0  0 41]]
Matriz de confusión de prueba para el modelo 3:
[[67  0  0]
 [ 1 28  0]
 [ 0  0 41]]


De aquí se puede concluir que existe una relación de compromiso entre clasificar correctamente las muestras de la primera y segunda clase. El límite de decisión que genera un error 0 requiera probablemente de mayor complejidad (trayendo aparejado un posible problema de overfitting) o siendo imposible de lograr.

El modelo 0 cuyos priors son los que más se asemejan a los reales, es uno de los que mejor performa. El modelo 3 da resultados similares; ello se debe a que la tercera clase es evidentemente es fácilmente separable de las demás (puesto a que todos los modelos, a pesar de la abismal diferencia entre priors, han sido capaz de clasificarlos correctamente) y por lo tanto, la información proporcionada por los datos es lo suficientemente fuerte como para compensar el sesgo del prior (análogo al dataset anterior).

### 3) Implementar LDA

In [10]:
from utils import BaseBayesianClassifier, inv, det

class LDA(BaseBayesianClassifier):
    # Utiliza una matriz de covarianza ponderada para evitar que clases con mayor cantidad de muestras tengan mayor influencia en su valor final. 
    # Al ponderar, se intenta que la variabilidad dentro de la clase más pequeña también sea tenida en cuenta de manera proporcional.

  def _fit_params(self, X, y):
    # Inicializa la matriz de covarianza ponderada
    
    cov_matrix = np.zeros((X.shape[0], X.shape[0]))
    
    # Suma la covarianza de cada clase ponderada por su tamaño
    for idx in range(len(self.log_a_priori)):
        X_class = X[:, y.flatten() == idx]
        cov_matrix += np.cov(X_class, bias=True) * X_class.shape[1] # estimador de máxima verosimilitud
        # cov_matrix += np.cov(X_class, bias=False) * (X_class.shape[1] - 1) # estimador insesgado

    # Divide por el número total de muestras
    cov_matrix /= X.shape[1] # estimador de máxima verosimilitud
    # # Divide por el número total de muestras (menos 1 para corrección de Bessel)
    # cov_matrix /= (X.shape[1] - len(self.log_a_priori)) # estimador insesgado
    
    # Calcula la inversa de la matriz de covarianza ponderada
    self.inv_cov = inv(cov_matrix)
    self.means = [X[:,y.flatten()==idx].mean(axis=1, keepdims=True) for idx in range(len(self.log_a_priori))]


  def _predict_log_conditional(self, x, class_idx):
    # predict the log(P(x|G=class_idx)), the log of the conditional probability of x given the class
    # this should depend on the model used
    unbiased_x =  x - self.means[class_idx]
    return 0.5*np.log(det(self.inv_cov)) -0.5 * unbiased_x.T @ self.inv_cov @ unbiased_x

Se comparar LDA vs QDA (Sin multiples prioris, es decir que se estimen a partir de los datos) con los dos datasets:

In [11]:
for dataset_name, dataset in zip(['iris', 'penguins'], [get_iris_dataset(), get_penguins()]):
    for model_name, curr_model in zip(['QDA', 'LDA'], [QDA, LDA]):
        model = curr_model()
        x_full, y_full = dataset
        train_x, train_y, test_x, test_y = split_transpose(x_full, y_full, 0.4, 6543)
        model.fit(train_x, train_y)
        train_acc = accuracy(train_y, model.predict(train_x))
        test_acc = accuracy(test_y, model.predict(test_x))
        print(f"[Dataset={dataset_name}][Model={model_name}] train err {1-train_acc:.4f}, test err {1-test_acc:.4f}")

[Dataset=iris][Model=QDA] train err 0.0111, test err 0.0167
[Dataset=iris][Model=LDA] train err 0.0222, test err 0.0167
[Dataset=penguins][Model=QDA] train err 0.0146, test err 0.0146
[Dataset=penguins][Model=LDA] train err 0.0098, test err 0.0146


### Conclusion

Según lo observado, LDA es un mejor modelo cuando se tiene de un número limitado de datos, mientras que QDA necesita de un tamaño mayor de datos y una mayor variabilidad entre clases para tener una mayor eficacia. Por lo tanto, en situaciones donde se cuenta con poca cantidad de datos o las clases no están bien representadas, LDA puede ser la mejor elección.

### 4) Utilizar otros 2 (dos) valores de *random seed* para obtener distintos splits de train y test, y repetir la comparación del punto anterior ¿Las conclusiones previas se mantienen?

In [12]:
import pandas as pd
df = pd.DataFrame()

for dataset_name,dataset in zip(['iris', 'penguins'],[get_iris_dataset(), get_penguins()]):
    for model_name, curr_model in zip(['QDA', 'LDA'],[QDA, LDA]):
        for seed in [6543, 5501,125]:
            model = curr_model()
            x_full, y_full = dataset
            train_x, train_y, test_x, test_y = split_transpose(x_full, y_full,test_sz=0.4, random_state=seed)
            model.fit(train_x, train_y)
            train_acc = accuracy(train_y, model.predict(train_x))
            test_acc = accuracy(test_y, model.predict(test_x))
            # print(f"[Dataset={dataset_name}][Model={model_name}] train err {1-train_acc:.4f}, test err {1-test_acc:.4f}")
            row = {
                'Dataset': dataset_name,
                'Model': model_name,
                'seed': seed,
                'Error (train)': 1-train_acc,
                'Error (test)': 1-test_acc,
            }
            
            df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
print(df)
    

     Dataset Model  seed  Error (train)  Error (test)
0       iris   QDA  6543       0.011111      0.016667
1       iris   QDA  5501       0.022222      0.016667
2       iris   QDA   125       0.022222      0.016667
3       iris   LDA  6543       0.022222      0.016667
4       iris   LDA  5501       0.022222      0.016667
5       iris   LDA   125       0.011111      0.016667
6   penguins   QDA  6543       0.014634      0.014599
7   penguins   QDA  5501       0.014634      0.007299
8   penguins   QDA   125       0.009756      0.014599
9   penguins   LDA  6543       0.009756      0.014599
10  penguins   LDA  5501       0.009756      0.014599
11  penguins   LDA   125       0.014634      0.007299


### Conclusion
Las conclusiones iniciales son válidas: LDA sigue siendo la mejor elección en situaciones con datos limitados o clases desbalanceadas, mientras que QDA podría beneficiarse de un mayor volumen de datos para maximizar su eficacia.

### 5) Tensorized QDA vs QDA

In [13]:
from utils import TensorizedQDA

x_full, y_full = get_iris_dataset()
train_x, train_y, test_x, test_y = split_transpose(x_full, y_full,test_sz=0.4, random_state=6543)

tqda = TensorizedQDA()
tqda.fit(train_x, train_y)
train_acc = accuracy(train_y, tqda.predict(train_x))
test_acc = accuracy(test_y, tqda.predict(test_x))

print(f"Train (apparent) error is {1-train_acc:.4f} while test error is {1-test_acc:.4f}")

Train (apparent) error is 0.0333 while test error is 0.0500


In [14]:
%%timeit

tqda.predict(test_x)

1.04 ms ± 53.5 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [15]:
tqda_predictions = tqda.predict(test_x)

In [16]:
qda = QDA()
qda.fit(train_x, train_y)
train_acc = accuracy(train_y, qda.predict(train_x))
test_acc = accuracy(test_y, qda.predict(test_x))
print(f"Train (apparent) error is {1-train_acc:.4f} while test error is {1-test_acc:.4f}")

Train (apparent) error is 0.0111 while test error is 0.0167


In [17]:
%%timeit

qda.predict(test_x)

2.72 ms ± 80.4 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Tensorized QDA aprovecha que varias de las operaciones que se tienen que realizar se pueden escribir de forma matricial, pudiendo realizarse de forma paralela. Esto se hace evidente en las siguientes líneas:

```
self.tensor_inv_cov = np.stack(self.inv_covs)
self.tensor_means = np.stack(self.means)
```

Lo cual muestra que las cuentas serán hechas simultáneamente, de forma paralela, para todas las clases y luego se verifica en la firma de la función:
```
_predict_log_conditionals(self, x)
```

Ya que no es necesario especificar para qué clase será hecha dicha predicción.

Además, es aproximadamente 3 veces más rápido que coincide con la cantidad de clases.

## Faster QDA

Si intentamos calcular todas las predicciones simultáneamente, terminamos calculando una matriz que contiene no solo la información de interés en la diagonal (auto-interacciones), sino también las interacciones cruzadas entre las observaciones.

La diagonal de esta matriz representa los términos que se usarían en el cálculo individual de la función discriminante para cada observación.
Los términos fuera de la diagonal representan interacciones entre diferentes observaciones, lo cual no es necesario si se quiere predecir para cada observación de forma independiente.
Por lo tanto, para obtener las predicciones, podrías ignorar los términos fuera de la diagonal, pero la construcción de toda la matriz sigue siendo costosa en términos de tiempo y memoria.

In [18]:
from utils import TensorizedQDA
class FasterQDAOptim1(TensorizedQDA):
    
    def _predict_log_conditionals(self, x):
        # k: clases
        # p: features
        # n: observaciones
        # x era de p x n
        # despues de hacer fit -> x es un tensor de k x p x n (o sea mismo array, k veces)
        # (se hace la prediccion para las k clases en conjunto)
        unbiased_x = x - self.tensor_means
        inner_prod = unbiased_x.transpose(0, 2, 1) @ self.tensor_inv_cov @ unbiased_x
        # print(inner_prod.shape) # matrices cuadradas que pide mostrar la consigna       DESCOMENTARRRRR!!!!!!
        # inner prod shape: (k, n, n) -> k matrices de n x n
        # se debe de pensar que como es un producto de tensores => cada producto produce una matriz de n x n
        
        # dado un k fijo (estando posicionados en una clase)
        # se puede imaginar que en cada matriz
        # obtengo diagonal,
        # ya que solo las que son consigo mismas son las que nos interesan (producto interno)
        new_mat = np.array([np.diag(mat) for mat in inner_prod]) # (k, n)
        # new_mat = np.sum(unbiased_x.transpose(0, 2, 1) @ self.tensor_inv_cov, axis=2)
        # new_mat = np.sum(self.tensor_inv_cov@unbiased_x, axis=1)
        itcov = np.log(det(self.tensor_inv_cov)) # (k,)
        
        # truco para no tener que crear un nuevo vector itcov de (k, n) para poder sumarlo
        return - 0.5 * itcov - 0.5 * new_mat.transpose() # (n, k)
        
    def predict(self, x):        
        log_cond = self._predict_log_conditionals(x) # (n, k)
        log_priors = self.log_a_priori #(k,)
        y_hat = self.encoder.names[np.argmax(log_priors + log_cond, axis=1)]
        
        return y_hat
    
fqda1 = FasterQDAOptim1()
fqda1.fit(train_x, train_y)
# fqda1.predict(train_x)

In [19]:
%%timeit
fqda1.predict(train_x)

54.2 μs ± 954 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [20]:
fqda1_predictions = fqda1.predict(test_x)
print(fqda1_predictions)

['versicolor' 'versicolor' 'virginica' 'setosa' 'virginica' 'virginica'
 'versicolor' 'versicolor' 'versicolor' 'virginica' 'virginica'
 'versicolor' 'virginica' 'setosa' 'setosa' 'versicolor' 'setosa' 'setosa'
 'versicolor' 'setosa' 'versicolor' 'setosa' 'virginica' 'setosa'
 'virginica' 'setosa' 'versicolor' 'setosa' 'versicolor' 'virginica'
 'versicolor' 'setosa' 'versicolor' 'virginica' 'virginica' 'versicolor'
 'virginica' 'versicolor' 'setosa' 'virginica' 'setosa' 'virginica'
 'setosa' 'setosa' 'versicolor' 'virginica' 'virginica' 'setosa' 'setosa'
 'setosa' 'virginica' 'virginica' 'versicolor' 'setosa' 'virginica'
 'setosa' 'versicolor' 'setosa' 'setosa' 'setosa']


In [21]:
from utils import TensorizedQDA
class FasterQDAOptim2(TensorizedQDA):
    
    # def _predict_log_conditionals(self, x):
    #     unbiased_x = x - self.tensor_means
    #     # bug en consigna corregido seria -> diag(A dot prod B) = sum cols (B @ A^t)
    #     new_mat = np.sum(self.tensor_inv_cov@unbiased_x, axis=1)
    #     itcov = np.log(det(self.tensor_inv_cov)) # (k,)
        
    #     # truco para no tener que crear un nuevo vector itcov de (k, n) para poder sumarlo
    #     return - 0.5 * itcov - 0.5 * new_mat.transpose() # (n, k)

    def _predict_log_conditionals(self, x):
        unbiased_x = x - self.tensor_means
        Z = (unbiased_x.transpose(0, 2, 1) @ self.tensor_inv_cov)
        new_mat = np.sum(Z.transpose(0, 2, 1) * unbiased_x, axis=1)
        itcov = np.log(det(self.tensor_inv_cov)) # (k,)
        
        # truco para no tener que crear un nuevo vector itcov de (k, n) para poder sumarlo
        return - 0.5 * itcov - 0.5 * new_mat.transpose() # (n, k)
        
    def predict(self, x):        
        log_cond = self._predict_log_conditionals(x) # (n, k)
        log_priors = self.log_a_priori #(k,)
        # print(log_priors + log_cond)
        y_hat = self.encoder.names[np.argmax(log_priors + log_cond, axis=1)]
        
        return y_hat
    
fqda2 = FasterQDAOptim2()
fqda2.fit(train_x, train_y)
# fqda2.predict(train_x)

In [22]:
%%timeit
fqda2.predict(train_x)

35.1 μs ± 965 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [23]:
fqda2_predictions = fqda2.predict(test_x)
print(fqda2_predictions)

['versicolor' 'versicolor' 'virginica' 'setosa' 'virginica' 'virginica'
 'versicolor' 'versicolor' 'versicolor' 'virginica' 'virginica'
 'versicolor' 'virginica' 'setosa' 'setosa' 'versicolor' 'setosa' 'setosa'
 'versicolor' 'setosa' 'versicolor' 'setosa' 'virginica' 'setosa'
 'virginica' 'setosa' 'versicolor' 'setosa' 'versicolor' 'virginica'
 'versicolor' 'setosa' 'versicolor' 'virginica' 'virginica' 'versicolor'
 'virginica' 'versicolor' 'setosa' 'virginica' 'setosa' 'virginica'
 'setosa' 'setosa' 'versicolor' 'virginica' 'virginica' 'setosa' 'setosa'
 'setosa' 'virginica' 'virginica' 'versicolor' 'setosa' 'virginica'
 'setosa' 'versicolor' 'setosa' 'setosa' 'setosa']


In [24]:
print(np.bincount(fqda1_predictions.flatten() == fqda2_predictions.flatten()))

[ 0 60]


In [25]:
print(np.bincount(tqda_predictions.flatten() == fqda2_predictions.flatten())) 

[ 0 60]


In [26]:
tqda._predict_custom(test_x)

[-181.45679406  -10.6300553   -20.62416842]
[-324.03689662   -6.68368449   -9.66714934]
[-405.02225118  -12.62608248   -6.54590058]
[ -10.14775866  -64.27396544 -122.70060224]
[-440.98215181  -13.8536638    -6.30540916]
[-360.1394166   -12.79236665  -11.94092015]
[-238.40035204   -6.91424733  -12.13844385]
[-237.79854061   -9.52686012  -17.83977562]
[-244.96608447   -6.14186958  -13.55483918]
[-664.41710353  -13.82814716   -6.36876014]
[-333.19020875  -11.70164296  -11.06726648]
[-255.41840509   -6.50105871  -13.59507457]
[-1022.25152484   -33.70506346   -16.75782211]
[ -8.44076956 -49.32264622 -99.99656945]
[  -7.2481765   -57.99254591 -114.82051969]
[-244.10653231   -8.45611458  -16.27830787]
[ -12.44343421  -87.0491007  -158.90538634]
[  -7.99677978  -57.41562616 -116.30408402]
[-310.74362622   -7.27247013  -15.70005565]
[ -8.09796106 -46.04022831 -94.36098035]
[-405.40734444   -7.37636432  -11.6792238 ]
[ -12.74062064  -54.63708988 -104.77179623]
[-482.37538442  -20.58207345   -7.1

array([['versicolor', 'versicolor', 'virginica', 'setosa', 'virginica',
        'virginica', 'versicolor', 'versicolor', 'versicolor',
        'virginica', 'virginica', 'versicolor', 'virginica', 'setosa',
        'setosa', 'versicolor', 'setosa', 'setosa', 'versicolor',
        'setosa', 'versicolor', 'setosa', 'virginica', 'setosa',
        'virginica', 'setosa', 'versicolor', 'setosa', 'versicolor',
        'virginica', 'versicolor', 'setosa', 'versicolor', 'virginica',
        'virginica', 'versicolor', 'virginica', 'versicolor', 'setosa',
        'virginica', 'setosa', 'virginica', 'setosa', 'setosa',
        'versicolor', 'virginica', 'virginica', 'setosa', 'setosa',
        'setosa', 'virginica', 'virginica', 'versicolor', 'setosa',
        'virginica', 'setosa', 'versicolor', 'setosa', 'setosa',
        'setosa']], dtype='<U10')

In [27]:
fqda2._predict_log_conditionals(test_x)

array([[ -181.45679406,   -10.6300553 ,   -20.62416842],
       [ -324.03689662,    -6.68368449,    -9.66714934],
       [ -405.02225118,   -12.62608248,    -6.54590058],
       [  -10.14775866,   -64.27396544,  -122.70060224],
       [ -440.98215181,   -13.8536638 ,    -6.30540916],
       [ -360.1394166 ,   -12.79236665,   -11.94092015],
       [ -238.40035204,    -6.91424733,   -12.13844385],
       [ -237.79854061,    -9.52686012,   -17.83977562],
       [ -244.96608447,    -6.14186958,   -13.55483918],
       [ -664.41710353,   -13.82814716,    -6.36876014],
       [ -333.19020875,   -11.70164296,   -11.06726648],
       [ -255.41840509,    -6.50105871,   -13.59507457],
       [-1022.25152484,   -33.70506346,   -16.75782211],
       [   -8.44076956,   -49.32264622,   -99.99656945],
       [   -7.2481765 ,   -57.99254591,  -114.82051969],
       [ -244.10653231,    -8.45611458,   -16.27830787],
       [  -12.44343421,   -87.0491007 ,  -158.90538634],
       [   -7.99677978,   -57.4

In [28]:
fqda1._predict_log_conditionals(test_x)

array([[ -181.45679406,   -10.6300553 ,   -20.62416842],
       [ -324.03689662,    -6.68368449,    -9.66714934],
       [ -405.02225118,   -12.62608248,    -6.54590058],
       [  -10.14775866,   -64.27396544,  -122.70060224],
       [ -440.98215181,   -13.8536638 ,    -6.30540916],
       [ -360.1394166 ,   -12.79236665,   -11.94092015],
       [ -238.40035204,    -6.91424733,   -12.13844385],
       [ -237.79854061,    -9.52686012,   -17.83977562],
       [ -244.96608447,    -6.14186958,   -13.55483918],
       [ -664.41710353,   -13.82814716,    -6.36876014],
       [ -333.19020875,   -11.70164296,   -11.06726648],
       [ -255.41840509,    -6.50105871,   -13.59507457],
       [-1022.25152484,   -33.70506346,   -16.75782211],
       [   -8.44076956,   -49.32264622,   -99.99656945],
       [   -7.2481765 ,   -57.99254591,  -114.82051969],
       [ -244.10653231,    -8.45611458,   -16.27830787],
       [  -12.44343421,   -87.0491007 ,  -158.90538634],
       [   -7.99677978,   -57.4

In [29]:
class TensorizedLDA(LDA):

    def _fit_params(self, X, y):
        # ask plain QDA to fit params
        super()._fit_params(X, y)
        self.fmt = y.dtype

        # stack onto new dimension
        self.tensor_inv_cov = np.stack(self.inv_cov)
        self.tensor_means = np.stack(self.means)
        self.unbiased_by_cov = self.tensor_means.transpose(0,2,1)@self.tensor_inv_cov

    def predict(self, x):        
        y_hat = self.encoder.names[np.argmax(self.unbiased_by_cov@(x-0.5*self.tensor_means), axis=0)] 
        return y_hat
    
    
tlda = TensorizedLDA()
tlda.fit(train_x, train_y)
tlda.predict(train_x)

array([['setosa', 'virginica', 'setosa', 'versicolor', 'versicolor',
        'virginica', 'versicolor', 'virginica', 'virginica',
        'versicolor', 'versicolor', 'virginica', 'setosa', 'setosa',
        'versicolor', 'virginica', 'virginica', 'virginica', 'virginica',
        'virginica', 'setosa', 'virginica', 'versicolor', 'versicolor',
        'versicolor', 'virginica', 'virginica', 'versicolor', 'setosa',
        'setosa', 'virginica', 'virginica', 'setosa', 'setosa',
        'versicolor', 'versicolor', 'virginica', 'setosa', 'setosa',
        'setosa', 'virginica', 'virginica', 'versicolor', 'setosa',
        'virginica', 'versicolor', 'virginica', 'setosa', 'virginica',
        'virginica', 'setosa', 'setosa', 'virginica', 'virginica',
        'virginica', 'versicolor', 'virginica', 'versicolor', 'setosa',
        'virginica', 'setosa', 'setosa', 'virginica', 'setosa',
        'virginica', 'virginica', 'versicolor', 'setosa', 'versicolor',
        'setosa', 'versicolor', 'ver

In [30]:
%%timeit
tlda.predict(train_x)

13.3 μs ± 1.01 μs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### queda Faster LDA por implementar

In [31]:
# %%timeit
# flda.predict(train_x)

# Preguntas Teoricas
### 1. 
Partimos de la probabilidad condicional $ P(x \mid y = j) $ para una clase $ j $, que en LDA sigue una distribución normal multivariada:

$$
f_j(x) = \frac{1}{(2\pi)^{d/2} |\Sigma|^{1/2}} \exp\left(-\frac{1}{2}(x - \mu_j)^T \Sigma^{-1} (x - \mu_j)\right)
$$

Donde:
   *  $x$  es un vector de características de dimensión  d ,
* $\mu_j$ es el vector de medias de la clase j ,
 * $\Sigma $ es la matriz de covarianza compartida entre las clases.

El logaritmo de esta función es:

$$
\log f_j(x) = \log \left( \frac{1}{(2\pi)^{d/2} |\Sigma|^{1/2}} \right) - \frac{1}{2} (x - \mu_j)^T \Sigma^{-1} (x - \mu_j)
$$

Este logaritmo se puede descomponer en dos términos:

$$
\log f_j(x) = -\frac{d}{2} \log(2\pi) - \frac{1}{2} \log |\Sigma| - \frac{1}{2} (x - \mu_j)^T \Sigma^{-1} (x - \mu_j)
$$

El primer término $ -\frac{d}{2} \log(2\pi) $ y el segundo término $ -\frac{1}{2} \log |\Sigma| $ son constantes con respecto a $ x $ y se pueden agrupar en una constante $ C' $:

$$
\log f_j(x) = -\frac{1}{2} (x - \mu_j)^T \Sigma^{-1} (x - \mu_j) + C'
$$

Luego, se distribuyen los terminos:

$$
\log f_j(x) = -\frac{1}{2} \left( x^T \Sigma^{-1} x - 2 \mu_j^T \Sigma^{-1} x + \mu_j^T \Sigma^{-1} \mu_j \right) + C'
$$

Distribuyendo el $ -\frac{1}{2} $:

$$
\log f_j(x) = -\frac{1}{2} x^T \Sigma^{-1} x + \mu_j^T \Sigma^{-1} x - \frac{1}{2} \mu_j^T \Sigma^{-1} \mu_j + C'
$$

Observamos que el término $ -\frac{1}{2} x^T \Sigma^{-1} x $ no depende de la clase $ j $, por lo que podemos absorberlo en la constante $ C' $, obteniendo:

$$
\log f_j(x) = \mu_j^T \Sigma^{-1} x - \frac{1}{2} \mu_j^T \Sigma^{-1} \mu_j + C'
$$

Finalmente, reordenando los términos:

$$
\log f_j(x) = \mu_j^T \Sigma^{-1} \left(x - \frac{1}{2} \mu_j\right) + C'
$$

## Demostración

$$
A =
\begin{pmatrix}
a_{11} & ... & a_{1p} \\
... & ... & ...\\
a_{n1} & ... & a_{np} \\
\end{pmatrix},
B = \begin{pmatrix}
b_{11} & ... & b_{1n} \\
... & ... & ...\\
b_{p1} & ... & b_{pn} \\
\end{pmatrix}
$$


$$
A.B =
\begin{pmatrix}
a_{11}b_{11} + ... + a_{1p}b_{p1} & ... & a_{11}b_{1n} + ... + a_{1p}b_{pn} \\
... & ... & ...\\
a_{n1}b_{11} + ... + a_{np}b_{p1} & ... & a_{n1}b_{1n} + ... + a_{np}b_{pn}\\
\end{pmatrix}
$$


$$
diag(A.B) = (a_{11}b_{11} + ... + a_{1p}b_{p1}) + ... + (a_{n1}b_{1n} + ... + a_{np}b_{pn})
$$


$$
diag(A.B) = \sum_{k=1}^p a_{1k}b_{k1} + ... + \sum_{k=1}^p a_{nk}b_{kn} = \sum_{j=1}^n \sum_{k=1}^p a_{jk}b_{kj}
$$


$$
np.sum(\sum_{k=1}^p a_{jk}b_{kj}, axis=1)
$$


$$
np.sum(A \odot B^T, axis=1)
$$



# Ejercicio Teorico
## Resolución

### Calculo de la salida de la red neuronal

Dada la red neuronal de dos capas con los parámetros iniciales:
$$
w^{(1)} = \begin{pmatrix}
0.1 & -0.5 \\
-0.3 & -0.9 \\
0.8 & 0.02
\end{pmatrix}, \quad
b^{(1)} = \begin{pmatrix}
0.1 \\
0.5 \\
0.8
\end{pmatrix}
$$

$$
w^{(2)} = \begin{pmatrix}
-0.4 & 0.2 & -0.5
\end{pmatrix}, \quad
b^{(2)} = 0.7
$$

Y la entrada $$ x = \begin{pmatrix} 1.8 \\ -3.4 \end{pmatrix} $$ 
Calculamos primero la salida de la primera capa

$$
z^{(1)} = w^{(1)} \cdot x + b^{(1)} = \begin{pmatrix}
0.1 & -0.5 \\
-0.3 & -0.9 \\
0.8 & 0.02
\end{pmatrix}
\begin{pmatrix}
1.8 \\
-3.4
\end{pmatrix}
+
\begin{pmatrix}
0.1 \\
0.5 \\
0.8
\end{pmatrix}
=
\begin{pmatrix}
1.98 \\
3.02 \\
2.172
\end{pmatrix}
$$

Luego, calculamos la activación de la primera capa usando la función sigmoidea:

$$
a^{(1)} = \sigma(z^{(1)}) = \sigma\left(\begin{pmatrix}
1.98 \\
3.02 \\
2.172
\end{pmatrix}\right) =
\begin{pmatrix}
0.8786 \\
0.9535 \\
0.8977
\end{pmatrix}
$$

La salida de la segunda capa es:

$$
z^{(2)} = w^{(2)} \cdot a^{(1)} + b^{(2)} = \begin{pmatrix}
-0.4 & 0.2 & -0.5
\end{pmatrix}
\begin{pmatrix}
0.8786 \\
0.9535 \\
0.8977
\end{pmatrix}
+ 0.7 = 0.0903
$$

Finalmente, la salida final de la red es:

$$
\hat{y} = \sigma(z^{(2)}) = \sigma(-0.3601) = 0.5226
$$


### Función de costo

La función de costo cuadrática es:

$$
J(\theta) = \frac{1}{2} (\hat{y} - y)^2 = \frac{1}{2} (0.5226 - 5)^2 = -4.4774
$$

### Backpropagation

#### Cálculo del error en la capa de salida

El error en la capa de salida es:

$$
\delta^{(2)} = \frac{\partial J}{\partial \hat{y}} = (\hat{y} - y) \cdot \sigma'(z^{(2)})
$$
Con $$ \sigma'(z) = \frac{e^{-z}}{(1 + e^{-z})^{2}} $$
Entonces:
$$
\delta^{(2)} = (0.5226 - 5) \cdot \sigma'(0.0903) = -1.1170
 $$
### Gradientes para la segunda capa

Las derivadas de la función de costo respecto a los pesos y sesgos de la segunda capa son:

$$
\frac{\partial J}{\partial w^{(2)}} = \delta^{(2)} \cdot (a^{(1)})^{T} =
 = \begin{pmatrix}
-0.9916 & -1.0650 & -1.0028
\end{pmatrix}
$$

$$
\frac{\partial J}{\partial b^{(2)}} = \delta^{(2)} = -1.1170
$$

#### Cálculo del error en la primera capa

El error en la primera capa es:

$$
\delta^{(1)} = (w^{(2)})^T \cdot \delta^{(2)} \cdot \sigma'(z^{(1)}) = =
\begin{pmatrix}
0.0476 \\
-0.0099 \\
0.0512
\end{pmatrix}
$$

#### Gradientes para la primera capa

Las derivadas de la función de costo respecto a los pesos y sesgos de la primera capa son:

$$
\frac{\partial J}{\partial w^{(1)}} = \delta^{(1)} \cdot (x)^T =  = \begin{pmatrix}
0.0857 & -0.1619 \\
-0.0178 & 0.00337 \\
0.0923 & -0.1743
\end{pmatrix}
$$

$$
\frac{\partial J}{\partial b^{(1)}} = \delta^{(1)} = \begin{pmatrix}
0.0476 \\
-0.0099 \\
0.0513
\end{pmatrix}
$$

### Conclusión

Los gradientes obtenidos para los pesos y sesgos de cada capa son:

$$
\frac{\partial J}{\partial w^{(1)}} = \begin{pmatrix}
0.0857 & -0.1619 \\
-0.0178 & 0.00337 \\
0.0923 & -0.1743
\end{pmatrix}, \quad
\frac{\partial J}{\partial b^{(1)}} = \begin{pmatrix}
0.0476 \\
-0.0099 \\
0.0513
\end{pmatrix}
$$

$$
\frac{\partial J}{\partial w^{(2)}} = \begin{pmatrix}
-0.9916 & -1.0650 & -1.0028
\end{pmatrix}, \quad
\frac{\partial J}{\partial b^{(2)}} = -1.1170
$$
